In [1]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

# TODO
- See if ticket and cabin have any importance
- Try data imputation instead of fillna

# Dataset

In [2]:
df = pd.read_csv('/home/arjun/Desktop/GitHub/Kaggle-competitions/_Datasets/Titanic/train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
y = df['Survived']

In [5]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [6]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [7]:
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})

/tmp/ipykernel_1031801/909058363.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
/tmp/ipykernel_1031801/909058363.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})


In [8]:
df.fillna(df.mean(), inplace=True)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,1.0
1,2,1,1,1,38.0,1,0,71.2833,2.0
2,3,1,3,1,26.0,0,0,7.9250,1.0
3,4,1,1,1,35.0,1,0,53.1000,1.0
4,5,0,3,0,35.0,0,0,8.0500,1.0


In [9]:
df_randomized = df.sample(frac=1).reset_index(drop=True)
train_df = df_randomized[:800]
test_df = df_randomized[800:]

In [10]:
test_df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
800,138,0,1,0,37.000000,1,0,53.1000,1.0
801,422,0,3,0,21.000000,0,0,7.7333,3.0
802,141,0,3,1,29.699118,0,2,15.2458,2.0
803,790,0,1,0,46.000000,0,0,79.2000,2.0
804,818,0,2,0,31.000000,1,1,37.0042,2.0
...,...,...,...,...,...,...,...,...,...
886,482,0,2,0,29.699118,0,0,0.0000,1.0
887,212,1,2,1,35.000000,0,0,21.0000,1.0
888,376,1,1,1,29.699118,1,0,82.1708,2.0
889,168,0,3,1,45.000000,1,4,27.9000,1.0


# Model Creation

In [22]:
num_epochs = 10000
lr =.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [12]:
class NeuralNet(nn.Module):
    def __init__(self, inp_size, h1, h2, h3,  out_size):
        super(NeuralNet, self).__init__()
        self.inp_size = inp_size
        self.lay1 = nn.Linear(inp_size, h1)
        self.lay2 = nn.ReLU()
        self.lay3 = nn.Linear(h1, h2)
        self.lay4 = nn.ReLU()
        self.lay5 = nn.Linear(h2, h3)
        self.lay6 = nn.ReLU()
        self.lay7 = nn.Linear(h3, out_size)
        self.drop1 = nn.Dropout(0.7)
        self.drop2 = nn.Dropout(0.7)
        
    def forward(self,x):
        out = self.lay1(x)
        out = self.lay2(out)
        out = self.drop1(out)
        out = self.lay3(out)
        out = self.lay4(out)
        out = self.lay5(out)
        out = self.drop2(out) # We don't apply sotmax the cross entropy loss will do that for us
        out = self.lay6(out)
        out = self.lay7(out)
        return out
    
model = NeuralNet(8,30,60,30,2).to(device)

In [13]:
# class NeuralNet(nn.Module):
#     def __init__(self, inp_size, h1, h2, out_size):
#         super(NeuralNet, self).__init__()
#         self.fc1 = nn.Linear(inp_size, h1)
#         self.bn1 = nn.BatchNorm1d(h1)
#         self.drop1 = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(h1, h2)
#         self.bn2 = nn.BatchNorm1d(h2)
#         self.drop2 = nn.Dropout(0.5)
#         self.fc3 = nn.Linear(h2, out_size)
        
#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.bn1(x)
#         x = nn.ReLU()(x)
#         x = self.drop1(x)
#         x = self.fc2(x)
#         x = self.bn2(x)
#         x = nn.ReLU()(x)
#         x = self.drop2(x)
#         x = self.fc3(x)
#         return x

# model = NeuralNet(8, 30, 15, 2).to(device)
    

In [14]:
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr = lr)

# Training Model

In [15]:
train_loader = DataLoader(dataset=torch.tensor(np.array(train_df)), shuffle=True, batch_size=1000)
test_loader = DataLoader(dataset=torch.tensor(np.array(test_df)), shuffle=True, batch_size=100)

In [23]:
for epoch in range(num_epochs):
    avg_loss = 0
    for i,batch in enumerate(train_loader):

        y = batch[:,1].to(device).long()
        X =torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)

        # Forward pass
        output = model(X)
        
        # Backward pass
        loss = lossCategory(output, y)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
        avg_loss += loss

    if not ((epoch+1)%100): print(f"Epoch:{epoch+1}/{num_epochs} Loss:{avg_loss/len(train_loader)}")

Epoch:100/1000 Loss:0.6503145098686218
Epoch:200/1000 Loss:0.6622473001480103
Epoch:300/1000 Loss:0.6453762054443359
Epoch:400/1000 Loss:0.6525751352310181
Epoch:500/1000 Loss:0.6453973650932312
Epoch:600/1000 Loss:0.6359815001487732
Epoch:700/1000 Loss:0.631661593914032
Epoch:800/1000 Loss:0.637162446975708
Epoch:900/1000 Loss:0.6319090723991394
Epoch:1000/1000 Loss:0.6312867999076843


# Checking Accuracy

In [17]:
correct = 0
total = 0
for i, batch in enumerate(train_loader):
    X = torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)
    y = batch[:, 1].to(device).long()
    output = model(X)
    softmax_output = F.softmax(output, dim=-1)
    _, y_pred = torch.max(softmax_output, dim=-1)
    correct += (y_pred == y).sum().item()
    total += y.size(0)
print('Accuracy:', correct / total * 100)
X.shape, y.shape, 

Accuracy: 62.375


(torch.Size([800, 8]), torch.Size([800]))

In [24]:
correct = 0
total = 0
for i, batch in enumerate(test_loader):
    X = torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)
    y = batch[:, 1].to(device).long()
    output = model(X)
    softmax_output = F.softmax(output, dim=-1)
    _, y_pred = torch.max(softmax_output, dim=-1)
    correct += (y_pred == y).sum().item()
    total += y.size(0)
print('Accuracy:', correct / total * 100)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x7 and 8x30)

# Inference on Test Dataset

In [19]:
df = pd.read_csv('/home/arjun/Desktop/GitHub/Kaggle-competitions/_Datasets/Titanic/test.csv')
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})
df.fillna(df.mean(), inplace=True)
ds = torch.tensor(np.array(df))
test_loader = torch.utils.data.DataLoader(dataset=ds, shuffle=False, batch_size=1)
ds.shape,df.shape

/tmp/ipykernel_1031801/2239982465.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
/tmp/ipykernel_1031801/2239982465.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})


(torch.Size([418, 8]), (418, 8))

In [20]:
l = []
id_ = []
for i,batch in enumerate(test_loader):
        X =batch.to(device).to(torch.float32)
        output = model(X)
        softmax_output = F.softmax(output, dim=-1)

        l.append(torch.argmax(softmax_output).item())
        id_.append(int(X[:,0].item()))

ans = pd.DataFrame({'PassengerId':id_,'Survived':l})
ans.to_csv('/home/arjun/Desktop/GitHub/Kaggle-competitions/_Datasets/Titanic/submission.csv', index=False)
print("Saved new O/P")
ans

Saved new O/P


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0


# Best Score: 0.79